# Analyzing borrowers’ risk of defaulting

We need to prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.


**Description of the data**

`children`: the number of children in the family

`days_employed`: how long the customer has been working

`dob_years`: the customer’s age

`education`: the customer’s education level

`education_id`: identifier for the customer’s education

`family_status`: the customer’s marital status

`family_status_id`: identifier for the customer’s marital status

`gender`: the customer’s gender

`income_type`: the customer’s income type

`debt`: whether the customer has ever defaulted on a loan

`total_income`: monthly income

`purpose`: reason for taking out a loan

## Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer

df = pd.read_csv('/datasets/credit_scoring_eng.csv')

df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [2]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,20.000000
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,401755.400475
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,75.000000
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000
total_income,19351.0,26787.568355,16475.450632,3306.762000,16488.504500,23202.870000,32549.611000,362496.645000


In [3]:
df.describe(include=['object']).T

,count,unique,top,freq
education,21525,15,secondary education,13750
family_status,21525,5,married,12380
gender,21525,3,F,14236
income_type,21525,8,employee,11119
purpose,21525,38,wedding ceremony,797


### Conclusion

Only 'days_employed' and 'total_income' have NaN values, and it's the same number, over 2000, about 10% of the dataset. It makes sense that someone who never worked will have no income, but we have to make sure this is the case before filling in zeros. 

We see there are negative values in "children" and "days_employed". This is most likely a typing error.

At least one borrower is 0 years old. This is probably a mistake of some sort.

The average defaulting probability is 8.1%.

## Data preprocessing

### Processing missing values

First we make sure the NaNs in "days_employed" match those in "total_income".

In [4]:
len(df[df['days_employed'].isna() & df['total_income'].notna()])

0

They do. Now we'll try to figure out who they are.

In [5]:
df[df['days_employed'].isna()]['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Most are employees, and the rest are likely to have an income as well. Therefore, the missing values are not zeros. We will assign median or mean values to all NaNs, but first we have to correct the negatives in "days_employed", and we might as well take care of the negative "children" at the same time.

In [6]:
df['days_employed'] = df['days_employed'].abs()
df['children'] = df['children'].apply('abs')

df[['days_employed', 'total_income']].agg(['count', 'mean', 'median'])

,days_employed,total_income
count,19351.000000,19351.000000
mean,66914.728907,26787.568355
median,2194.220567,23202.870000


Mean is significantly higher than the median for both, so we'll fill in the median based on education and income type.

In [7]:
df['days_employed'] = df['days_employed'].fillna(df.groupby(['education_id', 'income_type'])['days_employed']
                                                 .transform('median')
                                                )
df['total_income'] = df['total_income'].fillna(df.groupby(['education_id', 'income_type'])['total_income']
                                               .transform('median')
                                              )

print('NaN values in "days_employed":', df['days_employed'].isna().sum())
print('NaN values in "total_income":', df['total_income'].isna().sum())

NaN values in "days_employed": 0
NaN values in "total_income": 0


### Conclusion

Missing values are unknown, but are probably not zeros. Too many values were missing for deleting, so median values were filled in.

### Data type replacement

The "debt", with values of either 1 or 0, is obviously intended to be boolean. 
"days_employed" is likely supposed to be an integer. What is the meaning of the subdecimals then? Whatever it is, it's statistically insignificant, so we can safely round it to integers. No need to do that with "total_income" however, since you can get paid a fraction of most currencies.

In [8]:
df['debt'] = df['debt'].astype('bool')
df['days_employed'] = df['days_employed'].astype('int32')

df[['days_employed', 'debt']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 2 columns):
days_employed    21525 non-null int32
debt             21525 non-null bool
dtypes: bool(1), int32(1)
memory usage: 105.2 KB


The database info also showed us that all numeric values take up 64 bytes each - more than neccesary. We'll slim down the size of the dataframe by replacing their types as well.

In [9]:
for column in ['children', 'dob_years', 'education_id', 'family_status_id']:
    df[column] = df[column].astype('int8')
    
df['total_income'] = df['total_income'].astype('float32')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null float32
purpose             21525 non-null object
dtypes: bool(1), float32(1), int32(1), int8(4), object(5)
memory usage: 1.1+ MB


### Conclusion

Two columns were assigned another data type, and all numerical values were given a smaller version of their data types. Thanks to our treatment, the dataframe's size was reduced by almost half.

### Processing duplicates

First we check for duplicates.

In [10]:
print(f'Total duplicates: {df.duplicated().sum()}')
print(f'Percent of dataset: {df.duplicated().sum() / len(df):.1%}')

Total duplicates: 54
Percent of dataset: 0.3%


It's an insignificant percentage, so we can safely remove it.

In [11]:
df = df.drop_duplicates().reset_index(drop=True)

print(f'Total duplicates: {df.duplicated().sum()}')
print(f'Percent of dataset: {df.duplicated().sum() / len(df):.1%}')

Total duplicates: 0
Percent of dataset: 0.0%


### Conclusion

The dataset had an insignificant number of duplicates, and they were all removed.

### Categorizing Data

Let's have a look at the categorical value counts.

In [12]:
for column in ['education', 'family_status', 'gender', 'income_type', 'purpose']:
    print(column)
    print(df[column].value_counts())
    print()

education
secondary education    13705
bachelor's degree       4710
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        273
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

family_status
married              12344
civil partnership     4163
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

gender
F      14189
M       7281
XNA        1
Name: gender, dtype: int64

income_type
employee                       11091
business                        5080
retiree                         3837
civil servant                   1457
entrepreneur                       2
unemployed                         2
paternity / maternity leave     

We'll take care of these one by one. 
In "education" we need to change to lowercase.

In [13]:
df['education'] = df['education'].str.lower()
print(df['education'].value_counts())

secondary education    15188
bachelor's degree       5251
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64


Only 6 graduate degree holders. It's statistically insignificant, so we'll just combine them with the other degree holders.

In [14]:
df.loc[df['education'] == 'bachelor\'s degree' , 'education'] = 'academic degree'
df.loc[df['education'] == 'graduate degree' , 'education'] = 'academic degree'

print(df['education'].value_counts())

secondary education    15188
academic degree         5257
some college             744
primary education        282
Name: education, dtype: int64


"family_status" seems alright. Gender has a single unusual value. According to the data, females are twice as likely to take a loan, so we'll assume it's a female.

In [15]:
df.loc[df['gender'] == 'XNA' , 'gender'] = 'F'

print(df['gender'].value_counts())

F    14190
M     7281
Name: gender, dtype: int64


In "income_type" we have 4 values with 1-2 counts and we should merge them to the other categories based on educated guesses (even if we're wrong it's statistically insignificant). If the sole student have an income he's probably employed. The unemployed and those on paternity\maternity leave are getting welfare cheques based on the income they had as emploees, and will likely get back to be employed at some point. lastly, entrepreneurship is just a fancy word for business.

In [16]:
df.loc[df['income_type'] == 'student' , 'income_type'] = 'employee'
df.loc[df['income_type'] == 'unemployed' , 'income_type'] = 'employee'
df.loc[df['income_type'] == 'paternity / maternity leave' , 'income_type'] = 'employee'
df.loc[df['income_type'] == 'entrepreneur' , 'income_type'] = 'business'

df['income_type'].value_counts()

employee         11095
business          5082
retiree           3837
civil servant     1457
Name: income_type, dtype: int64

All purposes are related to education, property, wedding or buying a car. We'll add a column for purpose category based on that.

In [17]:
stemmer = SnowballStemmer('english')

test_vocab = ['wedding', 'car', 'automobile', 'property', 'house', 'housing', 
        'estate', 'university', 'education', 'educate', 'college',
       ]

for word in test_vocab:
    stem = stemmer.stem(word)
    print(stem)

wed
car
automobil
properti
hous
hous
estat
univers
educ
educ
colleg


In [18]:
def purpose_type(entry):
    for word in entry.split():
        stem = stemmer.stem(word)
        if stem == 'wed':
            return 'wedding'
        elif stem == 'car' or stem == 'automobil':
            return 'car'
        elif stem == 'educ' or stem == 'univers' or stem == 'colleg':
            return 'education'
        elif stem == 'properti' or stem == 'hous' or stem == 'estat':
            return 'property'
    return 'other'

df['purpose_type'] = df['purpose'].apply(purpose_type)
df['purpose_type'].value_counts()

property     10814
car           4308
education     4014
wedding       2335
Name: purpose_type, dtype: int64

There is something awkward about "children":

In [19]:
df['children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Seems unlikely that so many people had 20 children, considering nobody had 6-19 children. 2 is right above 0 in the numpad, so it's probably a repeating typo. 
We'll also add a boolean column for having children (any number). 

In [20]:
df.loc[df['children'] == 20, 'children'] = 2
df['children_bool'] = df['children'].astype('bool')

print(df['children'].value_counts())
print()
print(df['children_bool'].value_counts())


0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

False    14107
True      7364
Name: children_bool, dtype: int64


Last but not least, we'll add a column of income categories. We will need it later.  

In [21]:
df['total_income'].describe()

count     21471.000000
mean      26468.162109
std       15720.878906
min        3306.761963
25%       17211.000000
50%       22975.294922
75%       31726.048828
max      362496.656250
Name: total_income, dtype: float64

In [22]:
def income_level(income):
    if income < 0 or income == np.nan:
        return 'invalid'
    elif income <= 17000:
        return '0-17k'
    elif income <= 23000:
        return '17-23k'
    elif income <= 32000:
        return '23-32k'
    else:
        return '32k+'
    
df['income_level'] = df['total_income'].apply(income_level)
print(df['income_level'].value_counts())

17-23k    5520
23-32k    5475
32k+      5253
0-17k     5223
Name: income_level, dtype: int64


### Conclusion

Some categorical columns were messy, but we took care of that.

## Questions and Answers

- Is there a relation between having kids and repaying a loan on time?

In [23]:
df.pivot_table(values='debt', 
               index='children_bool', 
               aggfunc=['count', 'sum', 'mean'],
              )

,count,sum,mean
,debt,debt,debt
children_bool,,,
False,14107,1063.0,0.075353
True,7364,678.0,0.092070


### Conclusion

9.2% of borrowers with kids defaulted versus 7.5% of borrowers with no kids. Having kids makes you 22% more likley to default.

- Is there a relation between marital status and repaying a loan on time?

In [24]:
df.pivot_table(values='debt', 
               index='family_status', 
               aggfunc=['count', 'sum', 'mean'],
              ).sort_values(('mean', 'debt'), ascending=False)

,count,sum,mean
,debt,debt,debt
family_status,,,
unmarried,2810,274.0,0.097509
civil partnership,4163,388.0,0.093202
married,12344,931.0,0.075421
divorced,1195,85.0,0.071130
widow / widower,959,63.0,0.065693


### Conclusion

Marital status is correlated to defaulting. Unmarried and civil partnership have a higher than average probability to default, while all other have a lower than average chance, with widows\wodowers the most likely to repay on time.



- Is there a relation between income level and repaying a loan on time?

In [25]:
df.pivot_table(values='debt', 
               index='income_level', 
               aggfunc=['count', 'sum', 'mean'],
              ).sort_values(('mean', 'debt'), ascending=False)

,count,sum,mean
,debt,debt,debt
income_level,,,
17-23k,5520,503.0,0.091123
23-32k,5475,463.0,0.084566
0-17k,5223,409.0,0.078307
32k+,5253,366.0,0.069674


### Conclusion

Borrowers earning 17-23K a month are the most likely to default, followed by the 23-32K group. Surprisingly, those earning less than 17K are the more likely than average repay on time. Those earning 32K or more expectedly have the lowest chance to default.

- How do different loan purposes affect on-time repayment of the loan?

In [26]:
df.pivot_table(values='debt', 
               index='purpose_type', 
               aggfunc=['count', 'sum', 'mean'],
              ).sort_values(('mean', 'debt'), ascending=False)

,count,sum,mean
,debt,debt,debt
purpose_type,,,
car,4308,403.0,0.093547
education,4014,370.0,0.092177
wedding,2335,186.0,0.079657
property,10814,782.0,0.072314


### Conclusion

When buying a car, the borrower is least likely to repay on time, followed closely by paying for education. For wedding the probability of defaulting is slightly lower than average, and real estate is the safest category.

## General conclusion

In this report, we investigated data about loan takers, with the purpose of finding which factors correlate to repaying debt on time. During the analysis, we filled in gaps in the data, removed duplicates and arranged all borrowers in categories for easy comparison. We found that having kids, being unmarried or in civil partnership, earning 17-32K a month and borrowing to pay for a car or education are all related to higher than average chance of defaulting on your loan.